# Regresja wielokrotna 

- regresja wielokrotna
- jakość dopasowania

In [ ]:
# import numpy as np
# import pandas as pd

# import statsmodels.api as sm
# import statsmodels.formula.api as smf

# from scipy import stats
# from patsy import dmatrices
# from statsmodels.stats.stattools import durbin_watson

# import matplotlib.pyplot as plt

In [ ]:
# carseats = sm.datasets.get_rdataset(dataname="Carseats", package="ISLR", cache=True)

In [ ]:
# carseats_df = carseats.data

## Zadanie 1

1. Wyznacz liniową funkcję regresji opisującą zależność sprzedaży (`Sales`) od wysokości budżetu na reklamę (`Advertising`) i ceny (`Price`).


2. Zweryfikuj dopasowanie modelu `Sales ~ Advertising + Price`:
    
    a) Czy istnieje związek między zmienną zależną a przynajmniej jedną ze zmiennych niezależnych? Jeśli tak, to z którą (z którymi)?
       
    b) Czy wyraz wolny jest istotny w modelu?
       
    c) Oblicz i zinterpretuj wartość współczynnika determinacji $R^2$.
    
       
3. Wyznacz model liniowy najlepiej opisujący zależność sprzedaży fotelików samochodowych od pozostałych dostępnych zmiennych:
    
  a) Czy istnieje związek pomiędzy zmienną zależną a którąkolwiek ze zmiennych niezależnych?
       
  b) Dla których zmiennych niezależnych $X_j$, $j=1,\ldots,10$ można odrzucić hipotezę zerową  $H_0: \beta_j = 0$?
       
  c) Biorąc pod uwagę odpowiedź na poprzednie pytanie, wyznacz mniejszy model w oparciu tylko o zmienne, które faktycznie mają istotny wpływ na wielkość sprzedaży.
       
  d) Porównaj dopasowanie obu modeli (większego i mniejszego).
       
       
4. Podaj przewidywaną wartość sprzedaży dla przypadku:  
`pd.DataFrame({
    'Sales': np.NaN, 
    'CompPrice': [125.0], 
    'Income': [70.0], 
    'Advertising': [5.0], 
    'Population': [270.0], 
    'Price': [120.0],
    'ShelveLoc': ['Medium'], 
    'Age': [54.0], 
    'Education': [15.0], 
    'Urban': ['Yes'], 
    'US': ['Yes']
})`

5. Oceń jakość dopasowania.

In [ ]:
# model = smf.ols('Sales~Price+Advertising',data = carseats_df).fit()
# print(model.summary())

# print("Wartość statystyki testowej testu F (F-statistic): {}".format(model.fvalue))
# print("P-wartość testu F (Prob (F-statistic)): {}".format(model.f_pvalue))
# print("\n")


# print("Wartość statystki testu t-Studenta")
# print(model.tvalues)
# print("\n")

# print("P-wartości testu t-Studenta")
# print(model.pvalues)
# print("\n")


# print("Współczynnik determinacji R2 (R-squared): {}".format(model.rsquared))

In [ ]:
# columns = list(carseats_df.columns)
# columns.remove('Sales')
# features = "+".join(columns)
# print(features)
# model = smf.ols('Sales~'+features,data = carseats_df)
# fitted = model.fit()
# fitted.summary()

In [ ]:
# print("Wartość statystyki testowej testu F (F-statistic): {}".format(fitted.fvalue))
# print("P-wartość testu F (Prob (F-statistic)): {}".format(fitted.f_pvalue))
# print("\n")

# print("Wartość statystki testu t-Studenta")
# print(fitted.tvalues)
# print("\n")

# print("P-wartości testu t-Studenta")
# print(fitted.pvalues)

In [ ]:
# np.array(model.exog_names)[fitted.pvalues<0.05]

In [ ]:
# fitted_red = smf.ols('Sales~ShelveLoc+CompPrice+Income+Advertising+Price+Age',data = carseats_df).fit()
# fitted_red.summary()

In [ ]:
# print("Współczynnik determinacji R2 (R-squared): {}".format(fitted.rsquared))
# print("Współczynnik determinacji R2 (R-squared): {}".format(fitted_red.rsquared))

In [ ]:
# #predykcja
# new_obs = pd.DataFrame({ 'Sales': np.NaN, 'CompPrice': [125.0], 'Income': [70.0],
#               'Advertising': [5.0], 'Population': [270.0], 'Price': [120.0], 'ShelveLoc': ['Medium'], 'Age': [54.0],
#               'Education': [15.0], 'Urban': ['Yes'], 'US': ['Yes'] })

# fitted_red.predict(new_obs)

# Graficzna ewaluacja modeli regresji: 
- wykres wartości przewidywanych względem prawdziwych
- wykres residuów względem wartości dopasowanych

### Wykres wartości przewidywanych względem prawdziwych

$(y_i, \hat{y}_i)$

wykres wartości przewidywanych względem prawdziwych.

Przy idealnym dopasowaniu, punkty powinny się układać na prostej y=x.

In [ ]:
# Ylin = np.linspace(carseats_df['Sales'].min(),carseats_df['Sales'].max(),100)
# plt.scatter(carseats.data['Sales'],fitted_red.fittedvalues)
# plt.plot(Ylin,Ylin,color = 'red')
# plt.show()

## Wykres rezyduów:

1. $(i,e_i)$,

2. $(\hat{y}_i,e_i)$.

Oba wykresy powinny układać się wokół zera, bo błędy modelu mają średnią 0. Wspólna wariancja powinna się wyrażać tym, że żadna podgrupa punktów nie łapie większego odchylenia od prostej $y=0$

In [ ]:
# #1. (i,e_i)
# xrang = np.arange(1,len(fitted_red.resid)+1)
# plt.scatter(xrang,fitted_red.resid)
# plt.plot(xrang,np.repeat(0,len(xrang)),color = 'red')

In [ ]:
# #2. (\hat{y}_i,e_i)
# xrang = np.arange(fitted_red.fittedvalues.min(),fitted_red.fittedvalues.max(),0.1)
# plt.scatter(fitted_red.fittedvalues,fitted_red.resid)
# plt.plot(xrang,np.repeat(0,len(xrang)),color = 'red')

### Wykres kwantylowy dla reszt 

Do badania normalności błędów

(reszty teoretyczne vs. reszty empiryczne)

Wykres powinnien układać sie na prostej $y=x$.

In [ ]:
# import scipy as sp
# fig,ax = plt.subplots(figsize=(6,2.5))
# sp.stats.probplot(fitted_red.resid,plot = ax,dist = 'norm',fit = True)
# plt.show()

# Zadanie 2 

1. Wczytać dane `realest.csv`, które zawierają informacje na temat domów na przedmieściach Chicago: cena domu (`Price`), liczba sypialni (`Bedroom`), powierzchnia w stopach kwadratowych (`Space`), liczba pokoi (`Room`), szerokość frontu działki w stopach (`Lot`), roczny podatek od nieruchomości (`Tax`), liczba łazienek (`Bathroom`), liczba miejsc parkingowych w garażu (`Garage`) i stan domu (`Condition`, 0-dobry, 1-wymaga remontu).
       
2. Wyznacz model liniowy najlepiej opisujący zależność ceny domu od pozostałych zmiennych w zbiorze:
    
  a) Czy istnieje związek pomiędzy zmienną zależną a którąkolwiek ze zmiennych niezależnych?
       
  b) Dla których zmiennych niezależnych $X_j$, $j=1,\ldots,10$ można odrzucić hipotezę zerową  $H_0: \beta_j = 0$?
       
  c) Biorąc pod uwagę odpowiedź na poprzednie pytanie, wyznacz mniejszy model w oparciu tylko o zmienne, które faktycznie mają istotny wpływ na wielkość sprzedaży.
       
  d) Porównaj dopasowanie obu modeli (większego i mniejszego).
  
3. Jaki wpływ na cenę ma zwiększenie liczby sypialni o 1, 
 kiedy wartości wszystkich pozostałych zmiennych objaśniających są ustalone? 
 Znaleźć uzasadnienie tego pozornie błędnego wyniku. Porównać ten wynik  z wynikiem otrzymanym dla modelu linowegoopisującego zależność ceny domu jedynie od liczby sypialni.  

4. Podaj przewidywaną wartość sprzedaży dla przypadku:  
`pd.DataFrame({
    'Price': np.NaN, 
    'Bedroom': [3.0], 
    'Space': [1100], 
    'Room': [6.0], 
    'Lot': [32.0], 
    'Tax': [911.0],
    'Bathroom': [1.5], 
    'Garage': [1.0], 
    'Condition': [0.0]
})`

5. Oceń jakość dopasowania.

In [ ]:
# #1
# realest = pd.read_csv('realest.csv')
# print(realest)

In [ ]:
# #są Na wyrzućmy je
# realest = realest.dropna(axis =0)
# print(realest)

In [ ]:
# # model ze wszsystkimi zmiennymi
# columns = list(realest.columns)
# columns.remove('Price')
# features = "+".join(list(columns))

# model = smf.ols('Price~'+features,data=realest)
# fitted = model.fit()
# fitted.summary()

In [ ]:
# #3
# #model tylko ze zmienną Bedroom
# fitted_red2 = smf.ols('Price~Bedroom',data=realest).fit()
# fitted_red2.summary()